In [10]:
from pyomo.environ import SolverFactory
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
from pyomo.core import *
import pyomo.kernel as pmo
from numpy import isnan
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None
import matplotlib.dates as mdates
from numpy import isnan,average
import matplotlib.patches as mpatches

In [11]:
def typecheckconvert(x):
    if type(x)==dict:
        return x
    elif type(x)==pd.core.frame.DataFrame:
        if (x.shape)==(24,1):
            return dict(enumerate(x.iloc[:,0]))
        else:
            raise ValueError('Wrong datashape!! Recheck')
    elif type(x)==type(x)==pd.core.series.Series:
        if x.shape==(24,):
            return dict(enumerate(x.iloc[:]))
        else:
            raise ValueError('Wrong datashape!! Recheck')
    else:
        raise ValueError('Unsupported datatype!! Recheck')

def preprocess(df,location):
    df[f"Temp-{location}"]=df[f"Temp-{location}"].apply(pd.to_numeric, errors='coerce')
    days=int(len(df)/24)
    g2h_price=df["Spot Price"]
    g2h_emission=df["Spot Emission"]
    t_out=df[f"Temp-{location}"]
    solar=df[f"PV-{location}"]
    conf_period=pd.DataFrame(index=range(days*24),columns=range(1))
    avail=pd.DataFrame(index=range(days*24),columns=range(1))
    soc_usage=pd.DataFrame(index=range(days*24),columns=range(1))
    crttime=6
    for i in range(len(t_out)):
        if isnan(t_out.iloc[i]):
            t_out.iloc[i]=(t_out.iloc[i-1]+t_out.iloc[i+1])/2
    for i in range(days):
        for j in range(24):
            if j<crttime:
                conf_period.iloc[i*24+j,0]=0
                avail.iloc[i*24+j,0]=1
            elif j>=crttime and j<9:
                conf_period.iloc[i*24+j,0]=1
                avail.iloc[i*24+j,0]=1
            else:
                conf_period.iloc[i*24+j,0]=0
                if j<18:
                    avail.iloc[i*24+j,0]=0
                else:
                    avail.iloc[i*24+j,0]=1
            if j>=9 and j<18:
                soc_usage.iloc[i*24+j,0]=0.0171
            else:
                soc_usage.iloc[i*24+j,0]=0
    return t_out, g2h_price, g2h_emission, solar, conf_period, avail, soc_usage

def running_costs(df,t_out,year):
    days=int(len(df)/24)    
    el_cost=((df["Spot Price"]*1.24)/1000)+(0.4+4.841+2.794)/100 
    em_cost=(df["Spot Emission"])/1000
    g2h_price=df["Spot Price"]
    g2h_emission=df["Spot Emission"]
    avg_elec_1=0
    avg_elec_2=0
    avg_emi_1=0
    avg_emi_2=0
    avg_values=pd.DataFrame(index=range(days),columns=["Evenings average cost", "Evenings average emission", "Nights average cost", 
                                                            "Nights average emission"])
    for i in range(days):
        for j in range(24):
            if (j>17) and (j<22):
                avg_elec_1=avg_elec_1+el_cost.iloc[i*24+j]
                avg_emi_1=avg_emi_1+em_cost.iloc[i*24+j]
            if (j<6) or (j>20):
                avg_elec_2=avg_elec_2+el_cost.iloc[i*24+j]
                avg_emi_2=avg_emi_2+em_cost.iloc[i*24+j]
        avg_elec_1=avg_elec_1/4
        avg_emi_1=avg_emi_1/4
        avg_elec_2=avg_elec_2/9
        avg_emi_2=avg_emi_2/9
        avg_values["Evenings average cost"].iloc[i]=avg_elec_1
        avg_values["Nights average cost"].iloc[i]=avg_elec_2
        avg_values["Evenings average emission"].iloc[i]=avg_emi_1
        avg_values["Nights average emission"].iloc[i]=avg_emi_2
    cost_1=sum(avg_values["Evenings average cost"])
    cost_2=sum(avg_values["Nights average cost"])
    emission_1=sum(avg_values["Evenings average emission"])
    emission_2=sum(avg_values["Nights average emission"])
    t_in_ini=22
    sf=1
    h=1
    results=pd.DataFrame(columns=["G2H Tariff", "H2G Tariff", "G2H Emission", "H2G Emission", "To grid", "From grid", "P (EV)(kW)", "SOC EV(%)", 
                                "P (BESS)(kW)",  "SOC BESS(%)" , "Cost", "T_in", "T_out","P (H)(kW)","Wash","Dry","Dish"],
                        index=range(365*24))
    results["H2G Tariff"]=(df["Spot Price"]-2.5)/1000
    results["G2H Tariff"]=el_cost
    results["G2H Emission"]=em_cost/1000
    results["T_out"]=t_out
    results["H2G Emission"]=em_cost*0
    for i in range(days):
        model=concrete_model_onlyheat_nodr(t_in_ini,t_out.iloc[i*24:(i+1)*24],
                        g2h_price.iloc[i*24:(i+1)*24],g2h_emission.iloc[i*24:(i+1)*24],50,sf,h)
        solver=SolverFactory("gurobi")
        solver.solve(model)
        for t in range(24):
            results["P (EV)(kW)"].iloc[i*24+t]=0
            results["P (BESS)(kW)"].iloc[i*24+t]=0
            results["SOC EV(%)"].iloc[i*24+t]=0
            results["SOC BESS(%)"].iloc[i*24+t]=0
            results["To grid"].iloc[i*24+t]=0
            results["From grid"].iloc[i*24+t]=pmo.value(model.p_fromgrid[t]())
            results["T_in"].iloc[i*24+t]=pmo.value(model.t_in[t]())
            results["P (H)(kW)"].iloc[i*24+t]=pmo.value(model.HP_cons[t]())
            results["Wash"].iloc[i*24+t]=0
            results["Dry"].iloc[i*24+t]=0
            results["Dish"].iloc[i*24+t]=0
            results["Cost"].iloc[i*24+t]=results["G2H Tariff"].iloc[i*24+t]*results["From grid"].iloc[i*24+t]-results["H2G Tariff"].iloc[i*24+t]*results["To grid"].iloc[i*24+t]
        t_in_last=pmo.value(model.t_in[23]())-273
        t_in_ini=t_in_last
    results["T_in"]=results["T_in"]-273
    x=sum(results["Cost"])
    y=sum(results["G2H Emission"]*results["From grid"])
    print(f"Cost to run washing machine is € {round(cost_1*0.4038,2)} and emission is {round(emission_1*0.4038,2)} kg CO2")
    print(f"Cost to run tumble dryer is € {round(cost_1*0.6657,2)} and emission is {round(emission_1*0.6657,2)} kg CO2")
    print(f"Cost to run dishwasher is € {round(cost_1*0.57534,2)} and emission is {round(emission_1*0.57534,2)} kg CO2")
    print(f"Cost to charge EV is € {round(cost_2*10/0.95,2)} and emission is {round(emission_2/0.95*10,2)} kg CO2")
    print(f"Cost to run heat pump is € {round(x,2)} and emission is {round(y,2)} kg CO2")
    rng_1 = pd.date_range(f"01-01-{year} 00:00:00", f"31-12-{year} 23:00:00", freq='H')
    results_hp=results
    results_hp.index=rng_1
    rng = pd.date_range(f"01-01-{year} 00:00:00", f"31-12-{year} 23:00:00", freq='D')
    avg_values.index=rng
    return avg_values, results_hp

def concrete_model_onlyheat_nodr(
    ini_tin,
    t_out,
    spot_price,
    spot_emission,
    g2h_limit,
    sf,
    x_h):
    solver=SolverFactory("gurobi")
    model=ConcreteModel()
    horizon=list(range(24))
    model.T=Set(initialize=horizon, ordered=True)
    model.A=100 #m2
    model.t_ini=ini_tin+273
    #model.COP=2
    model.heat_loss_coeff=0.184
    model.Q_house=150*model.A/3600
    model.HP_cons=Var(model.T,within=NonNegativeReals)
    model.heat_loss=Var(model.T,within=Reals)
    model.T_ramp=1 #k
    model.heating_load=Var(model.T,within=NonNegativeReals)
    model.passive_cooling=Var(model.T,within=NonNegativeReals)
    model.g2h_average_price=(average(spot_price)*1.24+4+48.41+27.94)/1000
    model.h2g_average_price=average(spot_price)/1000
    model.g2h_average_emission=average(spot_emission)/1000
    model.h2g_average_emission=0
    spot_price=typecheckconvert(spot_price)
    spot_emission=typecheckconvert(spot_emission)
    t_out=t_out+273
    t_out=typecheckconvert(t_out)
    model.g2h_tariff=dict((k, (v*1.24+4+48.41+27.94)/1000) for k, v in spot_price.items())
    model.g2h_emission=dict((k, (v)/1000) for k, v in spot_emission.items())
    model.h2g_tariff=dict((k, (v-2.5)/1000) for k, v in spot_price.items())
    model.h2g_emission=dict((k, v*0) for k, v in spot_emission.items())
    model.COP=dict((k, (v-273)*0.0875+3.25) for k, v in t_out.items())
    model.g2h_limit=g2h_limit
    model.p_fromgrid=Var(model.T,within=PositiveReals)
    model.xp_grid=Var(model.T,within=pmo.Binary)
    model.t_in=Var(model.T, within=Reals)
    model.x_h=x_h
    #model.T_outlet=Var(model.T,within=NonNegativeReals)
    
    def power_constraint(model,t):
        return model.p_fromgrid[t]==(model.x_h*model.HP_cons[t])
    model.powerconstraint_7=Constraint(model.T,rule=power_constraint)
    
    def grid_purchase(model,t):
        return model.p_fromgrid[t]<=model.xp_grid[t]*model.g2h_limit
    model.powerconstraint_8=Constraint(model.T,rule=grid_purchase)
    
    def heatflow(model,t):
        if (t>0 and model.x_h==1):
            return model.t_in[t]==model.t_in[t-1]-model.passive_cooling[t]+model.heating_load[t]
        elif (t==0 and model.x_h==1):
            return model.t_in[t]==model.t_ini-model.passive_cooling[t]+model.heating_load[t]
        else:
            return model.t_in[t]==model.t_ini
    model.heatconstraint_1=Constraint(model.T,rule=heatflow)
    
    def heatlow(model,t):
        return model.t_in[t]>=294
    model.heatconstraint_2=Constraint(model.T,rule=heatlow)

    def heatmax(model,t):
        return model.t_in[t]<=295
    model.heatconstraint_3=Constraint(model.T,rule=heatmax)

    def heating(model,t):
        if model.x_h==1:
            return  model.heating_load[t]==model.HP_cons[t]*model.COP[t]/model.Q_house
        else:
            return Constraint.Skip
    model.heatconstraint_4=Constraint(model.T,rule=heating)
    
    def heat_loss(model,t):
        if (model.x_h==1 and t_out[t]<=295):
            return model.passive_cooling[t]==model.heat_loss_coeff*(model.t_in[t]-t_out[t])/model.Q_house
        elif (model.x_h==1 and t_out[t]>295):
            return model.passive_cooling[t]==0
        else:
            return Constraint.Skip
    model.heatcons_5=Constraint(model.T,rule=heat_loss) 
    
    def heat_ramp_up(model,t):
        if t==0:
            return model.t_in[t]-model.t_ini<=model.T_ramp
        else:
            return model.t_in[t]-model.t_in[t-1]<=model.T_ramp
    model.heatcons_6=Constraint(model.T,rule=heat_ramp_up)

    def heat_ramp_down(model,t):
        if t==0:
            return model.t_in[t]-model.t_ini>=-model.T_ramp
        else:
            return model.t_in[t]-model.t_in[t-1]>=-model.T_ramp
    model.heatcons_7=Constraint(model.T,rule=heat_ramp_down)
    
    def obj_rule(model):
        return (sum(((sf/model.g2h_average_price)*((model.g2h_tariff[t])*model.p_fromgrid[t]))
                    +((1-sf)/model.g2h_average_emission)*(model.g2h_emission[t]*model.p_fromgrid[t]) 
                    for t in horizon[:]))
    model.obj=Objective(rule=obj_rule, sense=minimize)
    
    return model


def concrete_model_1(
    bess_size,
    ev_bess_size,
    pch_bess,
    pds_bess,
    pch_ev,
    pds_ev,
    inisoc_bess,
    inisoc_ev,
    ini_tin,
    tar_soc,
    crt_soc,
    conf,
    t_out,
    solar,
    ev_avail,
    ev_use,
    spot_price,
    spot_emission,
    h2g_limit,
    g2h_limit,
    sf,
    x_wm,
    x_td,
    x_dw,
    x_h):
    solver=SolverFactory("gurobi")
    model=ConcreteModel()
    horizon=list(range(24))
    model.T=Set(initialize=horizon, ordered=True)
    model.E_ev=ev_bess_size
    model.E_bess=bess_size
    model.P_CH_bess=pch_bess
    model.P_DS_bess=pds_bess
    model.P_CH_ev=pch_ev
    model.P_DS_ev=pds_ev
    model.ini_soc_ev=inisoc_ev
    model.ini_soc_bess=inisoc_bess
    model.soc_max=1
    model.soc_min_ev=0.6
    model.soc_min_bess=0.2
    model.A=100 #m2
    model.t_ini=ini_tin+273
    #model.COP=2
    model.heat_loss_coeff=0.184
    model.Q_house=150*model.A/3600
    model.HP_cons=Var(model.T,within=Reals)
    model.heat_loss=Var(model.T,within=Reals)
    model.T_ramp=1 #k
    model.heating_load=Var(model.T,within=NonNegativeReals)
    model.passive_cooling=Var(model.T,within=NonNegativeReals)
    model.g2h_average_price=(average(spot_price)*1.24+4+48.41+27.94)/1000
    model.h2g_average_price=average(spot_price)/1000
    model.g2h_average_emission=average(spot_emission)/1000
    model.h2g_average_emission=0
    #model.solar={key: value * pv_size for key, value in solar_base.items()}
    model.solar=typecheckconvert(solar)
    spot_price=typecheckconvert(spot_price)
    spot_emission=typecheckconvert(spot_emission)
    conf=typecheckconvert(conf)
    ev_avail=typecheckconvert(ev_avail)
    ev_use=typecheckconvert(ev_use)
    t_out=t_out+273
    t_out=typecheckconvert(t_out)
    model.g2h_tariff=dict((k, (v*1.24+4+48.41+27.94)/1000) for k, v in spot_price.items())
    model.g2h_emission=dict((k, (v)/1000) for k, v in spot_emission.items())
    model.h2g_tariff=dict((k, (v-2.5)/1000) for k, v in spot_price.items())
    model.h2g_emission=dict((k, v*0) for k, v in spot_emission.items())
    model.COP=dict((k, (v-273)*0.0875+3.25) for k, v in t_out.items())
    model.h2g_limit=h2g_limit
    model.g2h_limit=g2h_limit
    model.conf=conf
    model.SOC_R=crt_soc
    model.xp_ev=Var(model.T,within=pmo.Binary)
    model.xp_bess=Var(model.T,within=pmo.Binary)
    model.p_ev=Var(model.T,within=Reals)
    model.p_pos_ev=Var(model.T,within=NonNegativeReals)
    model.p_neg_ev=Var(model.T,within=NonNegativeReals)
    model.soc_ev=Var(model.T, bounds=(0,1))
    model.p_bess=Var(model.T,within=Reals)
    model.p_pos_bess=Var(model.T,within=NonNegativeReals)
    model.p_neg_bess=Var(model.T,within=NonNegativeReals)
    model.soc_bess=Var(model.T, bounds=(0,1))
    model.p_fromgrid=Var(model.T,within=PositiveReals)
    model.p_togrid=Var(model.T,within=PositiveReals)
    model.xp_grid=Var(model.T,within=pmo.Binary)
    model.t_in=Var(model.T, within=Reals)
    model.xp_wash=Var(model.T,within=pmo.Binary)
    model.xp_dish=Var(model.T,within=pmo.Binary)
    model.xp_dry=Var(model.T,within=pmo.Binary)
    model.P_wash=0.4038
    model.P_dish=0.57534
    model.P_dry=0.6657
    model.x_wm=x_wm
    model.x_td=x_td
    model.x_dw=x_dw
    model.x_h=x_h
    #model.t_out=t_out
    def storageconservation_ev(model,t):
        if t==0:
            return model.soc_ev[t]==(model.ini_soc_ev+model.p_ev[t]*0.9/model.E_ev-ev_use[t])
        else:
            return model.soc_ev[t]==(model.soc_ev[t-1]+model.p_ev[t]*0.9/model.E_ev-ev_use[t])
    model.socconstraint_1=Constraint(model.T,rule=storageconservation_ev)
    
    def storageconservation_bess(model,t):
        if t==0:
            return model.soc_bess[t]==(model.ini_soc_bess+model.p_bess[t]*0.9/model.E_bess)
        else:
            return model.soc_bess[t]==(model.soc_bess[t-1]+model.p_bess[t]*0.9/model.E_bess)
    model.socconstraint_2=Constraint(model.T,rule=storageconservation_bess)
    
    def storagelowlimits_ev(model,t):
        return model.soc_ev[t]>=model.soc_min_ev
    model.socconstraint_3=Constraint(model.T,rule=storagelowlimits_ev)
    
    def storagelowlimits_bess(model,t):
        return model.soc_bess[t]>=model.soc_min_bess
    model.socconstraint_4=Constraint(model.T,rule=storagelowlimits_bess)
    
    def storagehighlimits_ev(model,t):
        return model.soc_ev[t]<=model.soc_max
    model.socconstraint_5=Constraint(model.T,rule=storagehighlimits_ev)
    
    def storagehighlimits_bess(model,t):
        return model.soc_bess[t]<=model.soc_max
    model.socconstraint_6=Constraint(model.T,rule=storagehighlimits_bess)
    
    def targetsoc_ev(model,t):
        if t==8:
            return model.soc_ev[t]>=tar_soc
        else:
            return Constraint.Skip
    model.socconstraint_7=Constraint(model.T,rule=targetsoc_ev)
    
    def socconfidence(model,t):
        return model.soc_ev[t]>=model.SOC_R*model.conf[t]
    model.socconstraint_8=Constraint(model.T,rule=socconfidence)
    
    def netcharging_ev(model,t):
        return model.p_ev[t]==model.p_pos_ev[t]-model.p_neg_ev[t]
    model.powerconstraint_1=Constraint(model.T,rule=netcharging_ev)
    
    def netcharging_bess(model,t):
        return model.p_bess[t]==model.p_pos_bess[t]-model.p_neg_bess[t]
    model.powerconstraint_2=Constraint(model.T,rule=netcharging_bess)
    
    def combipos_ev(model,t):
        return model.p_pos_ev[t]<=model.xp_ev[t]*model.P_CH_ev*ev_avail[t]
    model.powerconstraint_3=Constraint(model.T,rule=combipos_ev)
    
    def combipos_bess(model,t):
        return model.p_pos_bess[t]<=model.xp_bess[t]*model.P_CH_bess
    model.powerconstraint_4=Constraint(model.T,rule=combipos_bess)
    
    def combineg_ev(model,t):
        return model.p_neg_ev[t]<=(1-model.xp_ev[t])*model.P_DS_ev*ev_avail[t]
    model.powerconstraint_5=Constraint(model.T,rule=combineg_ev)
    
    def combineg_bess(model,t):
        return model.p_neg_bess[t]<=(1-model.xp_bess[t])*model.P_DS_bess
    model.powerconstraint_6=Constraint(model.T,rule=combineg_bess)
    
    def power_constraint(model,t):
        return model.solar[t]+model.p_neg_ev[t]+model.p_neg_bess[t]+model.p_fromgrid[t]==model.p_pos_ev[t]+model.p_pos_bess[t]+\
                model.p_togrid[t]+(model.x_h*model.HP_cons[t])+(model.xp_wash[t]*model.P_wash)+(model.xp_dry[t]*model.P_dry)+\
                (model.xp_dish[t]*model.P_dish)
    model.powerconstraint_7=Constraint(model.T,rule=power_constraint)
    
    def grid_purchase(model,t):
        return model.p_fromgrid[t]<=model.xp_grid[t]*model.g2h_limit
    model.powerconstraint_8=Constraint(model.T,rule=grid_purchase)
    
    def grid_sell(model,t):
        return model.p_togrid[t]<=(1-model.xp_grid[t])*model.h2g_limit
    model.powerconstraint_9=Constraint(model.T,rule=grid_sell)
    
    def heatflow(model,t):
        if (t>0 and model.x_h==1):
            return model.t_in[t]==model.t_in[t-1]-model.passive_cooling[t]+model.heating_load[t]
        elif (t==0 and model.x_h==1):
            return model.t_in[t]==model.t_ini-model.passive_cooling[t]+model.heating_load[t]
        else:
            return model.t_in[t]==model.t_ini
    model.heatconstraint_1=Constraint(model.T,rule=heatflow)
    
    def heatlow(model,t):
        return model.t_in[t]>=292
    model.heatconstraint_2=Constraint(model.T,rule=heatlow)

    def heatmax(model,t):
        return model.t_in[t]<=295
    model.heatconstraint_3=Constraint(model.T,rule=heatmax)

    def heating(model,t):
        if model.x_h==1:
            return  model.heating_load[t]==model.HP_cons[t]*model.COP[t]/model.Q_house
        else:
            return Constraint.Skip
    model.heatconstraint_4=Constraint(model.T,rule=heating)
    
    def heat_loss(model,t):
        if (model.x_h==1 and t_out[t]<=295):
            return model.passive_cooling[t]==model.heat_loss_coeff*(model.t_in[t]-t_out[t])/model.Q_house
        elif (model.x_h==1 and t_out[t]>295):
            return model.passive_cooling[t]==0
        else:
            return Constraint.Skip
    model.heatcons_5=Constraint(model.T,rule=heat_loss) 
    
    def heat_ramp_up(model,t):
        if t==0:
            return model.t_in[t]-model.t_ini<=model.T_ramp
        else:
            return model.t_in[t]-model.t_in[t-1]<=model.T_ramp
    model.heatcons_6=Constraint(model.T,rule=heat_ramp_up)

    def heat_ramp_down(model,t):
        if t==0:
            return model.t_in[t]-model.t_ini>=-model.T_ramp
        else:
            return model.t_in[t]-model.t_in[t-1]>=-model.T_ramp
    model.heatcons_7=Constraint(model.T,rule=heat_ramp_down)
    
    def wash(model):
        if model.x_wm==1:
            return sum(model.xp_wash[t] for t in model.T)==1
        else:
            return sum(model.xp_wash[t] for t in model.T)==0
    model.appconstraint_1=Constraint(rule=wash)

    def dry(model):
        if model.x_td==1:
            return sum(model.xp_dry[t] for t in model.T)==1
        else:
            return sum(model.xp_dry[t] for t in model.T)==0
    model.appconstraint_2=Constraint(rule=dry)

    def dish(model):
        if model.x_dw==1:
            return sum(model.xp_dish[t] for t in model.T)==1
        else:
            return sum(model.xp_dish[t] for t in model.T)==0
    model.appconstraint_3=Constraint(rule=dish)

    def washordry(model,t):
        return model.xp_wash[t]+model.xp_dry[t]<=1
    model.appconstraint_4=Constraint(model.T,rule=washordry) 

    def washbeforedry(model,t):
        if (t>0 and model.x_wm==1):
            return model.xp_dry[t]-sum(model.xp_wash[t] for t in range(0,t))<=0
        else:
            return Constraint.Skip
    model.appconstraint_5=Constraint(model.T,rule=washbeforedry)
    
    def washbeforenight(model,t):
        if (t==21 and model.x_wm==1):
            return sum(model.xp_wash[i] for i in range(0,t))==1
        else:
            return Constraint.Skip
    model.appconstraint_6=Constraint(model.T,rule=washbeforenight)
    
    def drybeforenight(model,t):
        if (t==21 and model.x_td==1):
            return sum(model.xp_dry[i] for i in range(0,t))==1
        else:
            return Constraint.Skip
    model.appconstraint_7=Constraint(model.T,rule=drybeforenight)
    
    def obj_rule(model):
        return (sum(((sf/model.g2h_average_price)*((model.g2h_tariff[t])*model.p_fromgrid[t]-(model.h2g_tariff[t])*model.p_togrid[t]))
                    +((1-sf)/model.g2h_average_emission)*((model.g2h_emission[t])*model.p_fromgrid[t]-model.h2g_emission[t]*model.p_togrid[t]) 
                    for t in horizon[:]))
    model.obj=Objective(rule=obj_rule, sense=minimize)
    
    return model



def concrete_model_ev_nopv_1(
    ev_bess_size,
    pch_ev,
    pds_ev,
    inisoc_ev,
    ini_tin,
    tar_soc,
    crt_soc,
    conf,
    t_out,
    ev_avail,
    ev_use,
    spot_price,
    spot_emission,
    h2g_limit,
    g2h_limit,
    sf,
    x_wm,
    x_td,
    x_dw,
    x_h):
    solver=SolverFactory("gurobi")
    model=ConcreteModel()
    horizon=list(range(24))
    model.T=Set(initialize=horizon, ordered=True)
    model.E_ev=ev_bess_size
    model.P_CH_ev=pch_ev
    model.P_DS_ev=pds_ev
    model.ini_soc_ev=inisoc_ev
    model.soc_max=1
    model.soc_min_ev=0.6
    model.A=100 #m2
    model.t_ini=ini_tin+273
    #model.COP=2
    model.heat_loss_coeff=0.184
    model.Q_house=150*model.A/3600
    model.HP_cons=Var(model.T,within=Reals)
    model.heat_loss=Var(model.T,within=Reals)
    model.T_ramp=1 #k
    model.heating_load=Var(model.T,within=NonNegativeReals)
    model.passive_cooling=Var(model.T,within=NonNegativeReals)
    model.g2h_average_price=(average(spot_price)*1.24+4+48.41+27.94)/1000
    model.h2g_average_price=average(spot_price)/1000
    model.g2h_average_emission=average(spot_emission)/1000
    model.h2g_average_emission=0
    spot_price=typecheckconvert(spot_price)
    spot_emission=typecheckconvert(spot_emission)
    conf=typecheckconvert(conf)
    ev_avail=typecheckconvert(ev_avail)
    ev_use=typecheckconvert(ev_use)
    t_out=t_out+273
    t_out=typecheckconvert(t_out)
    model.g2h_tariff=dict((k, (v*1.24+4+48.41+27.94)/1000) for k, v in spot_price.items())
    model.g2h_emission=dict((k, (v)/1000) for k, v in spot_emission.items())
    model.h2g_tariff=dict((k, (v-2.5)/1000) for k, v in spot_price.items())
    model.h2g_emission=dict((k, v*0) for k, v in spot_emission.items())
    model.COP=dict((k, (v-273)*0.0875+3.25) for k, v in t_out.items())
    model.h2g_limit=h2g_limit
    model.g2h_limit=g2h_limit
    model.conf=conf
    model.SOC_R=crt_soc
    model.xp_ev=Var(model.T,within=pmo.Binary)
    model.p_ev=Var(model.T,within=Reals)
    model.p_pos_ev=Var(model.T,within=NonNegativeReals)
    model.p_neg_ev=Var(model.T,within=NonNegativeReals)
    model.soc_ev=Var(model.T, bounds=(0,1))
    model.p_fromgrid=Var(model.T,within=PositiveReals)
    model.p_togrid=Var(model.T,within=PositiveReals)
    model.xp_grid=Var(model.T,within=pmo.Binary)
    model.t_in=Var(model.T, within=Reals)
    model.xp_wash=Var(model.T,within=pmo.Binary)
    model.xp_dish=Var(model.T,within=pmo.Binary)
    model.xp_dry=Var(model.T,within=pmo.Binary)
    model.P_wash=0.4038
    model.P_dish=0.57534
    model.P_dry=0.6657
    model.x_wm=x_wm
    model.x_td=x_td
    model.x_dw=x_dw
    model.x_h=x_h
    def storageconservation_ev(model,t):
        if t==0:
            return model.soc_ev[t]==(model.ini_soc_ev+model.p_ev[t]*0.9/model.E_ev-ev_use[t])
        else:
            return model.soc_ev[t]==(model.soc_ev[t-1]+model.p_ev[t]*0.9/model.E_ev-ev_use[t])
    model.socconstraint_1=Constraint(model.T,rule=storageconservation_ev)
    
    def storagelowlimits_ev(model,t):
        return model.soc_ev[t]>=model.soc_min_ev
    model.socconstraint_3=Constraint(model.T,rule=storagelowlimits_ev)
    
    def storagehighlimits_ev(model,t):
        return model.soc_ev[t]<=model.soc_max
    model.socconstraint_5=Constraint(model.T,rule=storagehighlimits_ev)
    
    def targetsoc_ev(model,t):
        if t==8:
            return model.soc_ev[t]>=tar_soc
        else:
            return Constraint.Skip
    model.socconstraint_7=Constraint(model.T,rule=targetsoc_ev)
    
    def socconfidence(model,t):
        return model.soc_ev[t]>=model.SOC_R*model.conf[t]
    model.socconstraint_8=Constraint(model.T,rule=socconfidence)
    
    def netcharging_ev(model,t):
        return model.p_ev[t]==model.p_pos_ev[t]-model.p_neg_ev[t]
    model.powerconstraint_1=Constraint(model.T,rule=netcharging_ev)
    
    def combipos_ev(model,t):
        return model.p_pos_ev[t]<=model.xp_ev[t]*model.P_CH_ev*ev_avail[t]
    model.powerconstraint_3=Constraint(model.T,rule=combipos_ev)
    
    def combineg_ev(model,t):
        return model.p_neg_ev[t]<=(1-model.xp_ev[t])*model.P_DS_ev*ev_avail[t]
    model.powerconstraint_5=Constraint(model.T,rule=combineg_ev)
    
    def power_constraint(model,t):
        return model.p_neg_ev[t]+model.p_fromgrid[t]==model.p_pos_ev[t]+model.p_togrid[t]+(model.x_h*model.HP_cons[t])+\
            (model.xp_wash[t]*model.P_wash)+(model.xp_dry[t]*model.P_dry)+(model.xp_dish[t]*model.P_dish)
    model.powerconstraint_7=Constraint(model.T,rule=power_constraint)
    
    def grid_purchase(model,t):
        return model.p_fromgrid[t]<=model.xp_grid[t]*model.g2h_limit
    model.powerconstraint_8=Constraint(model.T,rule=grid_purchase)
    
    def grid_sell(model,t):
        return model.p_togrid[t]<=(1-model.xp_grid[t])*model.h2g_limit
    model.powerconstraint_9=Constraint(model.T,rule=grid_sell)
    
    def heatflow(model,t):
        if (t>0 and model.x_h==1):
            return model.t_in[t]==model.t_in[t-1]-model.passive_cooling[t]+model.heating_load[t]
        elif (t==0 and model.x_h==1):
            return model.t_in[t]==model.t_ini-model.passive_cooling[t]+model.heating_load[t]
        else:
            return model.t_in[t]==model.t_ini
    model.heatconstraint_1=Constraint(model.T,rule=heatflow)
    
    def heatlow(model,t):
        return model.t_in[t]>=292
    model.heatconstraint_2=Constraint(model.T,rule=heatlow)

    def heatmax(model,t):
        return model.t_in[t]<=295
    model.heatconstraint_3=Constraint(model.T,rule=heatmax)

    def heating(model,t):
        if model.x_h==1:
            return  model.heating_load[t]==model.HP_cons[t]*model.COP[t]/model.Q_house
        else:
            return Constraint.Skip
    model.heatconstraint_4=Constraint(model.T,rule=heating)
    
    def heat_loss(model,t):
        if (model.x_h==1 and t_out[t]<=295):
            return model.passive_cooling[t]==model.heat_loss_coeff*(model.t_in[t]-t_out[t])/model.Q_house
        elif (model.x_h==1 and t_out[t]>295):
            return model.passive_cooling[t]==0
        else:
            return Constraint.Skip
    model.heatcons_5=Constraint(model.T,rule=heat_loss) 
    
    def heat_ramp_up(model,t):
        if t==0:
            return model.t_in[t]-model.t_ini<=model.T_ramp
        else:
            return model.t_in[t]-model.t_in[t-1]<=model.T_ramp
    model.heatcons_6=Constraint(model.T,rule=heat_ramp_up)

    def heat_ramp_down(model,t):
        if t==0:
            return model.t_in[t]-model.t_ini>=-model.T_ramp
        else:
            return model.t_in[t]-model.t_in[t-1]>=-model.T_ramp
    model.heatcons_7=Constraint(model.T,rule=heat_ramp_down)
    
    def wash(model):
        if model.x_wm==1:
            return sum(model.xp_wash[t] for t in model.T)==1
        else:
            return sum(model.xp_wash[t] for t in model.T)==0
    model.appconstraint_1=Constraint(rule=wash)

    def dry(model):
        if model.x_td==1:
            return sum(model.xp_dry[t] for t in model.T)==1
        else:
            return sum(model.xp_dry[t] for t in model.T)==0
    model.appconstraint_2=Constraint(rule=dry)

    def dish(model):
        if model.x_dw==1:
            return sum(model.xp_dish[t] for t in model.T)==1
        else:
            return sum(model.xp_dish[t] for t in model.T)==0
    model.appconstraint_3=Constraint(rule=dish)

    def washordry(model,t):
        return model.xp_wash[t]+model.xp_dry[t]<=1
    model.appconstraint_4=Constraint(model.T,rule=washordry) 

    def washbeforedry(model,t):
        if (t>0 and model.x_wm==1):
            return model.xp_dry[t]-sum(model.xp_wash[t] for t in range(0,t))<=0
        else:
            return Constraint.Skip
    model.appconstraint_5=Constraint(model.T,rule=washbeforedry)
    
    def washbeforenight(model,t):
        if (t==21 and model.x_wm==1):
            return sum(model.xp_wash[i] for i in range(0,t))==1
        else:
            return Constraint.Skip
    model.appconstraint_6=Constraint(model.T,rule=washbeforenight)
    
    def drybeforenight(model,t):
        if (t==21 and model.x_td==1):
            return sum(model.xp_dry[i] for i in range(0,t))==1
        else:
            return Constraint.Skip
    model.appconstraint_7=Constraint(model.T,rule=drybeforenight)
    
    def obj_rule(model):
        return (sum(((sf/model.g2h_average_price)*((model.g2h_tariff[t])*model.p_fromgrid[t]-model.h2g_tariff[t]*model.p_togrid[t]))
                    +((1-sf)/model.g2h_average_emission)*(model.g2h_emission[t]*model.p_fromgrid[t]-model.h2g_emission[t]*model.p_togrid[t]) 
                    for t in horizon[:]))
    model.obj=Objective(rule=obj_rule, sense=minimize)

    return model



In [23]:
year=2019
path=r"/Users/z102933/Desktop/PhD/Data/ABM/"
df=pd.read_excel(rf'{path}2019_data_upload.xlsx')
t_out, g2h_price, g2h_emission, solar, conf_period, avail, soc_usage=preprocess(df,"Helsinki")

In [13]:
#SET INPUT PARAMETERS
year=2019
days=int(len(df)/24)
soc_ev_ini=0.6
soc_bess_ini=0.2
t_in_ini=20.32
sf=1
wm=1
dw=1
td=1
h=1
p_ch=p_ds=3.6
tar_soc=0.85
crt_soc=0.7
bess=20
ev=65
h2g_limit=17
g2h_limit=17

In [14]:
avg_running_cost,results_hp=running_costs(df,t_out,year)
avg_running_cost.to_pickle(rf"{path}{year}_avgrunningcosts_upload.pkl")
results_hp.to_pickle(rf"{path}/2019_results_hp_upload.pkl")

Cost to run washing machine is € 26.59 and emission is 19.82 kg CO2
Cost to run tumble dryer is € 43.84 and emission is 32.68 kg CO2
Cost to run dishwasher is € 37.89 and emission is 28.24 kg CO2
Cost to charge EV is € 536.26 and emission is 439.35 kg CO2
Cost to run heat pump is € 969.52 and emission is 0.84 kg CO2


In [15]:
#PV+BESS
results=pd.DataFrame(columns=["G2H Tariff", "H2G Tariff", "G2H Emission", "H2G Emission", "To grid", "From grid", "P (EV)(kW)", "SOC EV(%)", 
                              "P (BESS)(kW)",  "SOC BESS(%)" , "P (Solar)(kW)","Cost", "Emission", "T_in", "T_out","P (H)(kW)","Wash","Dry","Dish"],
                     index=range(days*24))
results["H2G Tariff"]=(g2h_price-2.5)/1000
results["G2H Tariff"]=(g2h_price*1.24+4+48.41+27.94)/1000
results["G2H Emission"]=g2h_emission/1000
results["T_out"]=t_out
results["H2G Emission"]=g2h_emission*0
results["P (Solar)(kW)"]=solar
for i in range(days):
    model=concrete_model_1(bess,ev,p_ch,p_ds,p_ch,p_ds,soc_bess_ini,soc_ev_ini,t_in_ini,tar_soc,crt_soc,conf_period.iloc[i*24:(i+1)*24,0],
                           t_out.iloc[i*24:(i+1)*24],solar.iloc[i*24:(i+1)*24],avail.iloc[i*24:(i+1)*24,0],
                           soc_usage.iloc[i*24:(i+1)*24,0],g2h_price.iloc[i*24:(i+1)*24],g2h_emission.iloc[i*24:(i+1)*24],h2g_limit,
                           g2h_limit,sf,wm,td,dw,0)
    
    solver=SolverFactory("gurobi")
    solver.solve(model)
    for t in range(24):
        results["P (EV)(kW)"].iloc[i*24+t]=pmo.value(model.p_ev[t]())
        results["P (BESS)(kW)"].iloc[i*24+t]=pmo.value(model.p_bess[t]())
        results["SOC EV(%)"].iloc[i*24+t]=pmo.value(model.soc_ev[t]())
        results["SOC BESS(%)"].iloc[i*24+t]=pmo.value(model.soc_bess[t]())
        results["To grid"].iloc[i*24+t]=pmo.value(model.p_togrid[t]())
        results["From grid"].iloc[i*24+t]=pmo.value(model.p_fromgrid[t]())
        results["T_in"].iloc[i*24+t]=pmo.value(model.t_in[t]())
        results["P (H)(kW)"].iloc[i*24+t]=pmo.value(model.HP_cons[t]())
        results["Wash"].iloc[i*24+t]=abs(pmo.value(model.xp_wash[t]()))
        results["Dry"].iloc[i*24+t]=abs(pmo.value(model.xp_dry[t]()))
        results["Dish"].iloc[i*24+t]=abs(pmo.value(model.xp_dish[t]()))
        results["Cost"].iloc[i*24+t]=results["G2H Tariff"].iloc[i*24+t]*results["From grid"].iloc[i*24+t]-results["H2G Tariff"].iloc[i*24+t]*\
            results["To grid"].iloc[i*24+t]
    soc_ev_last=pmo.value(model.soc_ev[23]())
    soc_bess_last=pmo.value(model.soc_bess[23]())
    t_in_last=pmo.value(model.t_in[23]())-273
    soc_bess_ini=soc_bess_last
    soc_ev_ini=soc_ev_last
    t_in_ini=t_in_last
results["T_in"]=results["T_in"]-273
results["Emission"]=results["From grid"]*results["G2H Emission"]
rng = pd.date_range(f"01-01-{year} 00:00:00", f"31-12-{year} 23:00:00", freq='H')
results.index=rng

In [16]:
results_nohp_pv=results
results_nohp_pv.to_pickle(rf"{path}{year}_results_nohp_pv_upload.pkl")

In [17]:
#PV+BESS, HP
results=pd.DataFrame(columns=["G2H Tariff", "H2G Tariff", "G2H Emission", "H2G Emission", "To grid", "From grid", "P (EV)(kW)", "SOC EV(%)", 
                              "P (BESS)(kW)",  "SOC BESS(%)" , "P (Solar)(kW)","Cost", "Emission", "T_in", "T_out","P (H)(kW)","Wash","Dry","Dish"],
                     index=range(days*24))
results["H2G Tariff"]=(g2h_price-2.5)/1000
results["G2H Tariff"]=(g2h_price*1.24+4+48.41+27.94)/1000
results["G2H Emission"]=g2h_emission/1000
results["T_out"]=t_out
results["H2G Emission"]=g2h_emission*0
results["P (Solar)(kW)"]=solar
for i in range(days):
    model=concrete_model_1(bess,ev,p_ch,p_ds,p_ch,p_ds,soc_bess_ini,soc_ev_ini,t_in_ini,tar_soc,crt_soc,conf_period.iloc[i*24:(i+1)*24,0],
                           t_out.iloc[i*24:(i+1)*24],solar.iloc[i*24:(i+1)*24],avail.iloc[i*24:(i+1)*24,0],
                           soc_usage.iloc[i*24:(i+1)*24,0],g2h_price.iloc[i*24:(i+1)*24],g2h_emission.iloc[i*24:(i+1)*24],h2g_limit,
                           g2h_limit,sf,wm,td,dw,1)
    
    solver=SolverFactory("gurobi")
    solver.solve(model)
    for t in range(24):
        results["P (EV)(kW)"].iloc[i*24+t]=pmo.value(model.p_ev[t]())
        results["P (BESS)(kW)"].iloc[i*24+t]=pmo.value(model.p_bess[t]())
        results["SOC EV(%)"].iloc[i*24+t]=pmo.value(model.soc_ev[t]())
        results["SOC BESS(%)"].iloc[i*24+t]=pmo.value(model.soc_bess[t]())
        results["To grid"].iloc[i*24+t]=pmo.value(model.p_togrid[t]())
        results["From grid"].iloc[i*24+t]=pmo.value(model.p_fromgrid[t]())
        results["T_in"].iloc[i*24+t]=pmo.value(model.t_in[t]())
        results["P (H)(kW)"].iloc[i*24+t]=pmo.value(model.HP_cons[t]())
        results["Wash"].iloc[i*24+t]=abs(pmo.value(model.xp_wash[t]()))
        results["Dry"].iloc[i*24+t]=abs(pmo.value(model.xp_dry[t]()))
        results["Dish"].iloc[i*24+t]=abs(pmo.value(model.xp_dish[t]()))
        results["Cost"].iloc[i*24+t]=results["G2H Tariff"].iloc[i*24+t]*results["From grid"].iloc[i*24+t]-results["H2G Tariff"].iloc[i*24+t]*\
            results["To grid"].iloc[i*24+t]
    soc_ev_last=pmo.value(model.soc_ev[23]())
    soc_bess_last=pmo.value(model.soc_bess[23]())
    t_in_last=pmo.value(model.t_in[23]())-273
    soc_bess_ini=soc_bess_last
    soc_ev_ini=soc_ev_last
    t_in_ini=t_in_last
results["T_in"]=results["T_in"]-273
results["Emission"]=results["From grid"]*results["G2H Emission"]
rng = pd.date_range(f"01-01-{year} 00:00:00", f"31-12-{year} 23:00:00", freq='H')
results.index=rng

In [18]:
results_hp_pv=results
results_hp_pv.to_pickle(rf"{path}{year}_results_hp_pv_upload.pkl")

In [19]:
#HP
#sf=1
results=pd.DataFrame(columns=["G2H Tariff", "H2G Tariff", "G2H Emission", "H2G Emission", "To grid", "From grid", "P (EV)(kW)", "SOC EV(%)", 
                              "P (BESS)(kW)",  "SOC BESS(%)" , "Cost", "Emission", "T_in", "T_out", "P (H)(kW)","Wash","Dry","Dish"],
                     index=range(days*24))
results["H2G Tariff"]=(g2h_price-2.5)/1000
results["G2H Tariff"]=(g2h_price*1.24+4+48.41+27.94)/1000
results["G2H Emission"]=g2h_emission/1000
results["T_out"]=t_out
results["H2G Emission"]=g2h_emission*0
for i in range(days):
    model=concrete_model_ev_nopv_1(ev,p_ch,p_ds,soc_ev_ini,t_in_ini,tar_soc,crt_soc,conf_period.iloc[i*24:(i+1)*24,0],
                                 t_out.iloc[i*24:(i+1)*24],avail.iloc[i*24:(i+1)*24,0],
                                 soc_usage.iloc[i*24:(i+1)*24,0],g2h_price.iloc[i*24:(i+1)*24],g2h_emission.iloc[i*24:(i+1)*24],
                                 h2g_limit,g2h_limit,sf,wm,td,dw,1)
    
    solver=SolverFactory("gurobi")
    solver.solve(model)
    for t in range(24):
        results["P (EV)(kW)"].iloc[i*24+t]=pmo.value(model.p_ev[t]())
        #results["P (BESS)(kW)"].iloc[i*24+t]=pmo.value(model.p_bess[t]())
        results["P (BESS)(kW)"].iloc[i*24+t]=0
        results["SOC EV(%)"].iloc[i*24+t]=pmo.value(model.soc_ev[t]())
        #results["SOC BESS(%)"].iloc[i*24+t]=pmo.value(model.soc_bess[t]())
        results["SOC BESS(%)"].iloc[i*24+t]=0
        results["To grid"].iloc[i*24+t]=pmo.value(model.p_togrid[t]())
        results["From grid"].iloc[i*24+t]=pmo.value(model.p_fromgrid[t]())
        results["T_in"].iloc[i*24+t]=pmo.value(model.t_in[t]())
        results["P (H)(kW)"].iloc[i*24+t]=pmo.value(model.HP_cons[t]())
        results["Wash"].iloc[i*24+t]=pmo.value(model.xp_wash[t]())
        results["Dry"].iloc[i*24+t]=pmo.value(model.xp_dry[t]())
        results["Dish"].iloc[i*24+t]=pmo.value(model.xp_dish[t]())
        results["Cost"].iloc[i*24+t]=results["G2H Tariff"].iloc[i*24+t]*results["From grid"].iloc[i*24+t]-results["H2G Tariff"].iloc[i*24+t]*\
            results["To grid"].iloc[i*24+t]
    soc_ev_last=pmo.value(model.soc_ev[23]())
    #soc_bess_last=pmo.value(model.soc_bess[23]())
    t_in_last=pmo.value(model.t_in[23]())-273
    #soc_bess_ini=soc_bess_last
    soc_ev_ini=soc_ev_last
    t_in_ini=t_in_last
results["T_in"]=results["T_in"]-273
results["Emission"]=results["From grid"]*results["G2H Emission"]
rng = pd.date_range(f"01-01-{year} 00:00:00", f"31-12-{year} 23:00:00", freq='H')
results.index=rng

In [20]:
results_hp_nopv=results
results_hp_nopv.to_pickle(rf"{path}{year}_results_hp_nopv_upload.pkl")

In [21]:
#Only APP and EV
#sf=1
results=pd.DataFrame(columns=["G2H Tariff", "H2G Tariff", "G2H Emission", "H2G Emission", "To grid", "From grid", "P (EV)(kW)", "SOC EV(%)", 
                              "P (BESS)(kW)",  "SOC BESS(%)" , "Cost", "Emission", "T_in", "T_out", "P (H)(kW)","Wash","Dry","Dish"],
                     index=range(days*24))
results["H2G Tariff"]=(g2h_price-2.5)/1000
results["G2H Tariff"]=(g2h_price*1.24+4+48.41+27.94)/1000
results["G2H Emission"]=g2h_emission/1000
results["T_out"]=t_out
results["H2G Emission"]=g2h_emission*0
for i in range(days):
    model=concrete_model_ev_nopv_1(ev,p_ch,p_ds,soc_ev_ini,t_in_ini,tar_soc,crt_soc,conf_period.iloc[i*24:(i+1)*24,0],
                                 t_out.iloc[i*24:(i+1)*24],avail.iloc[i*24:(i+1)*24,0],
                                 soc_usage.iloc[i*24:(i+1)*24,0],g2h_price.iloc[i*24:(i+1)*24],g2h_emission.iloc[i*24:(i+1)*24],
                                 h2g_limit,g2h_limit,sf,wm,td,dw,0)
    
    solver=SolverFactory("gurobi")
    solver.solve(model)
    for t in range(24):
        results["P (EV)(kW)"].iloc[i*24+t]=pmo.value(model.p_ev[t]())
        #results["P (BESS)(kW)"].iloc[i*24+t]=pmo.value(model.p_bess[t]())
        results["P (BESS)(kW)"].iloc[i*24+t]=0
        results["SOC EV(%)"].iloc[i*24+t]=pmo.value(model.soc_ev[t]())
        #results["SOC BESS(%)"].iloc[i*24+t]=pmo.value(model.soc_bess[t]())
        results["SOC BESS(%)"].iloc[i*24+t]=0
        results["To grid"].iloc[i*24+t]=pmo.value(model.p_togrid[t]())
        results["From grid"].iloc[i*24+t]=pmo.value(model.p_fromgrid[t]())
        results["T_in"].iloc[i*24+t]=pmo.value(model.t_in[t]())
        results["P (H)(kW)"].iloc[i*24+t]=0
        results["Wash"].iloc[i*24+t]=pmo.value(model.xp_wash[t]())
        results["Dry"].iloc[i*24+t]=pmo.value(model.xp_dry[t]())
        results["Dish"].iloc[i*24+t]=pmo.value(model.xp_dish[t]())
        results["Cost"].iloc[i*24+t]=results["G2H Tariff"].iloc[i*24+t]*results["From grid"].iloc[i*24+t]-results["H2G Tariff"].iloc[i*24+t]*\
            results["To grid"].iloc[i*24+t]
    soc_ev_last=pmo.value(model.soc_ev[23]())
    #soc_bess_last=pmo.value(model.soc_bess[23]())
    t_in_last=pmo.value(model.t_in[23]())-273
    #soc_bess_ini=soc_bess_last
    soc_ev_ini=soc_ev_last
    t_in_ini=t_in_last
results["T_in"]=results["T_in"]-273
results["Emission"]=results["From grid"]*results["G2H Emission"]
rng = pd.date_range(f"01-01-{year} 00:00:00", f"31-12-{year} 23:00:00", freq='H')
results.index=rng

In [22]:
results_nohp_nopv=results
results_nohp_nopv.to_pickle(rf"{path}{year}_results_nohp_nopv_upload.pkl")